In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from scipy import spatial
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data= pd.read_csv('Books.csv', encoding="utf-8")
data

,Unnamed: 0,_id,author,contents,publishing house,year
0,0,Jane Eyre_Feltrinelli_pulito.txt,Charlotte Brontë,Quel giorno era impossibile uscire a passeggio...,Feltrinelli Editore,2014
1,1,Jane Eyre_Treves_pulito.txt,Charlotte Brontë,In quel giorno era impossibile passeggiare. La...,Fratelli Treves Editori,1904


In [3]:
data['documents_cleaned']= data.apply(lambda row: nltk.sent_tokenize(row['contents']), axis=1)
data

,Unnamed: 0,_id,author,contents,publishing house,year,documents_cleaned
0,0,Jane Eyre_Feltrinelli_pulito.txt,Charlotte Brontë,Quel giorno era impossibile uscire a passeggio...,Feltrinelli Editore,2014,[Quel giorno era impossibile uscire a passeggi...
1,1,Jane Eyre_Treves_pulito.txt,Charlotte Brontë,In quel giorno era impossibile passeggiare. La...,Fratelli Treves Editori,1904,"[In quel giorno era impossibile passeggiare., ..."


In [4]:
import numpy as np
documents = list(np.array(data['documents_cleaned'][0]))
documents2 = list(np.array(data['documents_cleaned'][1]))

df = pd.DataFrame(documents,columns={'text'})
df2 = pd.DataFrame(documents2, columns={'text'})
df3 = df.append(df2)

#WORD2VEC

In [5]:
corpus_text = 'n'.join(data['contents'])
data = []
# iterate through each sentence in the file
for i in sent_tokenize(corpus_text):
    temp = []
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)

In [6]:
# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count=5,
                                 window=10,
                                 size=50,
                                 sample=0,
                                 batch_words=100) 
# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 50, window = 5, sg = 1)

In [7]:
#model2.train(data, total_examples=model2.corpus_count, epochs=60, report_delay=1)

In [8]:
model1.init_sims(replace=True)
model1.wv.most_similar(positive=["giorno"])

[('momento', 0.876094400882721),
 ('grido', 0.8738731145858765),
 ('periodo', 0.8493399024009705),
 ('sapore', 0.8353150486946106),
 ('terreno', 0.8338131904602051),
 ('rumore', 0.8256413340568542),
 ('matrimonio', 0.8248665928840637),
 ('postiglione', 0.8212254643440247),
 ('mortale', 0.8203728795051575),
 ('posto', 0.8196715116500854)]

In [9]:
model2.init_sims(replace=True)

In [10]:
model2.wv.most_similar(positive=["giorno"])

[('mattino', 0.8813042640686035),
 ("un'ora", 0.8781667351722717),
 ('pomeriggio', 0.8759865164756775),
 ('mattina', 0.8722423911094666),
 ('periodo', 0.8673176765441895),
 ('durante', 0.8646053671836853),
 ('viaggio', 0.8634263277053833),
 ('seguente', 0.863265872001648),
 ('anno', 0.8567696213722229),
 ('ascoltare', 0.8549370169639587)]

In [11]:
model2.wv.similarity('giorno','mattino')

0.8813043

In [12]:
similar = model2.wv.most_similar('mattino')
similar

[('pomeriggio', 0.9795013666152954),
 ('villaggio', 0.9643508791923523),
 ('buio', 0.9559431672096252),
 ('cominciò', 0.9519914388656616),
 ('viaggio', 0.9508246779441833),
 ('campanello', 0.9483917951583862),
 ('sonno', 0.9483866691589355),
 ('capo', 0.9454476833343506),
 ('paesaggio', 0.9451088905334473),
 ('suono', 0.9444425106048584)]

In [13]:
model2.init_sims(replace = True)
model2.save('word2vec_model')
model = Word2Vec.load('word2vec_model')
model.wv.most_similar('pomeriggio')

[('mattino', 0.9795013666152954),
 ('campanello', 0.962501049041748),
 ('buio', 0.9547379612922668),
 ('tornò', 0.954379677772522),
 ('salotto', 0.9542637467384338),
 ('seguì', 0.9518462419509888),
 ('villaggio', 0.9514014720916748),
 ('capo', 0.9507164359092712),
 ('cominciò', 0.9506141543388367),
 ('viaggio', 0.9497316479682922)]

In [14]:
model.wv.most_similar('mattino')

[('pomeriggio', 0.9795013666152954),
 ('villaggio', 0.9643508791923523),
 ('buio', 0.9559431672096252),
 ('cominciò', 0.9519914984703064),
 ('viaggio', 0.9508246779441833),
 ('campanello', 0.9483917951583862),
 ('sonno', 0.9483866691589355),
 ('capo', 0.9454476833343506),
 ('paesaggio', 0.9451088905334473),
 ('suono', 0.9444427490234375)]

In [15]:
distance = model.wv.wmdistance(data[0],data[12])
distance

0.7093478444329921

In [16]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'
distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.04292970875905756

In [17]:
sentence_a = 'Ne ero contenta.'
sentence_b = 'Ne fui felice.'
distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.4715757810708749

In [18]:
sentence_a = 'Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'
sentence_b ='Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'

distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.15958282491764356

In [19]:
sentence_a = 'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

sentence_b = 'Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '

distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.07314634555238063

#DOC2VEC

In [20]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(df3.text)]
model_d2v = Doc2Vec(vector_size=5, dbow_words=1, dm=0, epochs=30,  window=2, seed=1337, min_count=1, alpha=0.001, min_alpha=0.025)
model_d2v.build_vocab(tagged_data)
model_d2v.save('doc2vec_model')

In [21]:
model_d2v = Doc2Vec(vector_size=5, window=2, min_count=1, workers=4)
model_d2v.build_vocab(tagged_data)

In [22]:
tokens = 'In quel giorno era impossibile passeggiare.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(8070, 0.9971870183944702),
 (13446, 0.9925793409347534),
 (9267, 0.9874575138092041),
 (245, 0.987050473690033),
 (19638, 0.986670196056366),
 (17626, 0.9866509437561035),
 (2214, 0.9840645790100098),
 (8470, 0.9832953810691833),
 (7285, 0.9827989339828491),
 (5705, 0.97891765832901)]

In [78]:
tagged_data[8070]

TaggedDocument(words=['Si', 'asciugò', 'gli', 'occhi', 'con', 'il', 'grembiule', '.'], tags=[8070])

In [79]:
tagged_data[13446]

TaggedDocument(words=['Leah', 'tornò', 'subito', ',', 'seguita', 'dalla', 'signora', 'Fairfax', ',', 'che', 'mi', 'ripetè', 'la', 'storia', '.'], tags=[13446])

In [25]:
tokens = 'Quel giorno era impossibile uscire a passeggio.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(8244, 0.9931377172470093),
 (8555, 0.9898731708526611),
 (5638, 0.9792236685752869),
 (4094, 0.9789928197860718),
 (15286, 0.9755794405937195),
 (15477, 0.9746443629264832),
 (1956, 0.9731001853942871),
 (17774, 0.9714725017547607),
 (15471, 0.9699475169181824),
 (5691, 0.9676898121833801)]

In [80]:
tagged_data[8244]

TaggedDocument(words=['``', 'Avrà', 'avuto', 'strane', 'disavventure', ',', 'immagino', '...', 'povera', 'vagabonda', 'pallida', 'ed', 'emaciata', '!', "''"], tags=[8244])

In [81]:
tagged_data[8555]

TaggedDocument(words=['Diana', 'si', 'offrì', "d'insegnarmi", 'il', 'tedesco', '.'], tags=[8555])

In [28]:
tokens = 'In quel giorno era impossibile passeggiare.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(8070, 0.9971870183944702),
 (13446, 0.9925793409347534),
 (9267, 0.9874575138092041),
 (245, 0.987050473690033),
 (19638, 0.986670196056366),
 (17626, 0.9866509437561035),
 (2214, 0.9840645790100098),
 (8470, 0.9832953810691833),
 (7285, 0.9827989339828491),
 (5705, 0.97891765832901)]

In [82]:
tagged_data[8070]

TaggedDocument(words=['Si', 'asciugò', 'gli', 'occhi', 'con', 'il', 'grembiule', '.'], tags=[8070])

In [30]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.01030303690006517

In [86]:
sentence_a = 'Ne ero contenta.'
sentence_b = 'Ne fui felice.'

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.058982812350404215

In [87]:
sentence_a = 'Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'
sentence_b ='Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.03019871237199146

In [88]:
sentence_a = 'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

sentence_b = 'Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.009897180821247182

In [31]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'
new_vector_a = model_d2v.infer_vector(sentence_a)
new_vector_b = model_d2v.infer_vector(sentence_b)

cosine_similarity = np.dot(new_vector_a, new_vector_b)/(np.linalg.norm(new_vector_a)* np.linalg.norm(new_vector_b))
cosine_similarity

-0.12602656

In [32]:
score = model_d2v.docvecs.most_similar([model_d2v.infer_vector(data[0])],topn=20)
score

[(5521, 0.9973555207252502),
 (18697, 0.9838452339172363),
 (3298, 0.9830261468887329),
 (9402, 0.976330578327179),
 (4918, 0.9757598638534546),
 (14006, 0.9753888845443726),
 (15897, 0.9729633927345276),
 (16999, 0.9718747138977051),
 (3386, 0.9700525403022766),
 (12091, 0.9691222906112671),
 (7548, 0.9671382904052734),
 (5158, 0.9650741219520569),
 (10745, 0.9649518132209778),
 (18784, 0.9649115800857544),
 (8516, 0.9643202424049377),
 (105, 0.961357593536377),
 (9988, 0.96097332239151),
 (6575, 0.9593194127082825),
 (1737, 0.9580193161964417),
 (13467, 0.9576738476753235)]

In [33]:
tagged_data[12100]

TaggedDocument(words=['Il', 'dopopranzo', 'trascorse', 'nella', 'pace', 'e', "nell'armonia", '.'], tags=[12100])

In [34]:
import numpy as np
def cos_sim(a, b):
  dot_product = np.dot(a, b)
  norm_a = np.linalg.norm(a)
  norm_b = np.linalg.norm(b)
  return dot_product / (norm_a * norm_b)


print(cos_sim(new_vector_a,new_vector_b))

-0.12602656


In [35]:
cos_distance = spatial.distance.cosine(new_vector_a, new_vector_b)
cos_distance

1.1260265707969666

In [36]:
#Euclidean similarity
euclidean_similarity = 1 / (1 + (np.linalg.norm(new_vector_a-new_vector_b)))
euclidean_similarity

0.8262653249972245

#SENT2VEC

In [37]:
!pip install bert
!pip install pytorch_pretrained_bert
!pip install sent2vec
!pip install sentence_transformers

In [38]:
import tensorflow as tf
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import random
import numpy as np
import matplotlib.pyplot as plt
import math


In [39]:
from sentence_transformers import SentenceTransformer, util
#model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
model = SentenceTransformer('distiluse-base-multilingual-cased')

In [40]:
# Two lists of sentences
sentences1 = documents

sentences2 = documents2

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

In [41]:
#Output the pairs with their score
for i in range(len(sentences1[:30])):
      print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[0], sentences2[i], cosine_scores[i][i]))

Quel giorno era impossibile uscire a passeggio. 		 In quel giorno era impossibile passeggiare. 		 Score: 0.9490
Quel giorno era impossibile uscire a passeggio. 		 La mattina avevamo errato per un'ora nel boschetto spogliato di foglie, ma dopo pranzo (quando non vi erano invitati, la signora Reed desinava presto), il vento gelato d'inverno aveva portato seco nubi così scure e una pioggia così penetrante, che non si poteva pensare a nessuna escursione. 		 Score: 0.8784
Quel giorno era impossibile uscire a passeggio. 		 Ne ero contenta. 		 Score: 0.9129
Quel giorno era impossibile uscire a passeggio. 		 Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed. 		 Score: 0.3790
Quel giorno era impossibile uscire a passeggio. 

In [42]:
document_embedding = model.encode(data)

In [43]:
query_embedding = model.encode(['Quel giorno era impossibile uscire a passeggio.'])
passage_embedding = model.encode(['In quel giorno era impossibile passeggiare.'])

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(query_embedding, passage_embedding)
cosine_scores

tensor([[0.9490]])

In [44]:
query_embedding = model.encode(['Ne fui felice.'])

passage_embedding = model.encode(['Ne ero contenta.'])
#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(query_embedding, passage_embedding)
cosine_scores

tensor([[0.9129]])

In [45]:
query_embedding = model.encode(['Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'])

passage_embedding = model.encode(['Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '])
#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(query_embedding, passage_embedding)
cosine_scores

tensor([[0.8191]])

In [46]:
query_embedding = model.encode(['Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'])
passage_embedding = model.encode(['Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'])
#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(query_embedding, passage_embedding)
cosine_scores

tensor([[0.7208]])

In [47]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = documents

sentences2 = documents2

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

In [48]:
#Output the pairs with their score
for i in range(len(sentences1[:30])):
      print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[0], sentences2[i], cosine_scores[i][i]))

Quel giorno era impossibile uscire a passeggio. 		 In quel giorno era impossibile passeggiare. 		 Score: 0.9375
Quel giorno era impossibile uscire a passeggio. 		 La mattina avevamo errato per un'ora nel boschetto spogliato di foglie, ma dopo pranzo (quando non vi erano invitati, la signora Reed desinava presto), il vento gelato d'inverno aveva portato seco nubi così scure e una pioggia così penetrante, che non si poteva pensare a nessuna escursione. 		 Score: 0.7308
Quel giorno era impossibile uscire a passeggio. 		 Ne ero contenta. 		 Score: 0.5094
Quel giorno era impossibile uscire a passeggio. 		 Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed. 		 Score: 0.6617
Quel giorno era impossibile uscire a passeggio. 

In [49]:
import torch#pytorch
from transformers import AutoTokenizer, AutoModel#for embeddings
from sklearn.metrics.pairwise import cosine_similarity#for similarity

#download pretrained model
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")
model = AutoModel.from_pretrained("dbmdz/bert-base-italian-cased",output_hidden_states=True)

Some weights of the model checkpoint at dbmdz/bert-base-italian-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [50]:
#create embeddings
def get_embeddings(text,token_length):
  tokens=tokenizer(text,max_length=token_length,padding='max_length',truncation=True)
  output=model(torch.tensor(tokens.input_ids).unsqueeze(0),
               attention_mask=torch.tensor(tokens.attention_mask).unsqueeze(0)).hidden_states[-1]
  return torch.mean(output,axis=1).detach().numpy()

In [51]:
#calculate similarity
def calculate_similarity(text1,text2,token_length=20):
    out1=get_embeddings(text1,token_length=token_length)#create embeddings of text
    out2=get_embeddings(text2,token_length=token_length)#create embeddings of text
    sim= cosine_similarity(out1,out2)[0][0]
    print(sim)

In [52]:
calculate_similarity(documents[0],documents2[0])

0.95590216


In [53]:
sentence_a = 'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

sentence_b = 'Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '

calculate_similarity(sentence_a,sentence_b)

0.80446357


In [83]:
sentence_a = 'Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'
sentence_b ='Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'

calculate_similarity(sentence_a,sentence_b)

0.94164723


In [84]:
sentence_a = 'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

sentence_b = 'Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '

calculate_similarity(sentence_a,sentence_b)

0.80446357


In [85]:
sentence_a = 'Ne ero contenta.'
sentence_b = 'Ne fui felice.'

calculate_similarity(sentence_a,sentence_b)

0.9024403


In [54]:
sent = list(i for i in documents)

In [55]:
sent = list(i for i in df3.text)

In [56]:
embedding_bert = []
for i in sent:
  output_bert=get_embeddings(i,token_length=20)#create embeddings of text
  embedding_bert.append(output_bert)

In [57]:
embed = []
for i in documents:
  out1=get_embeddings(i,token_length=20)#create embeddings of text
  embed.append(out1)

In [58]:
embed2 = []
for i in documents2:
  out2=get_embeddings(i,token_length=20)#create embeddings of text
  embed2.append(out2)

In [59]:
cosine_similarity(embed[0],embed2[0])

array([[0.95590216]], dtype=float32)

In [74]:
cosine_similarity(embed[2],embed2[2])

array([[0.9024403]], dtype=float32)

In [75]:
cosine_similarity(embed[3],embed2[3])

array([[0.7896439]], dtype=float32)

In [76]:
documents[3]

'Le lunghe passeggiate non mi erano mai piaciute, specie nei pomeriggi gelidi.'

In [77]:
documents2[3]

'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

In [61]:
import torch
torch.save(model, 'embedding_bert')
embedding_bert = torch.load('embedding_bert')